**<노트북 경로 설정>**
1. 개인 노트북용: C:/Users/82109/Desktop/Data Science/Competition/2020_Bigcontest_working/dataset
2. 학원 노트북용: C:/Users/Playdata/2020_Bigcontest_working/dataset/

# (윤성님이 보내주신) "홈쇼핑+시청률" 데이터 셋에 "기상" 데이터 결합

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from tqdm.notebook import tqdm

In [2]:
# 불필요한 첫 번째 컬럼(Unnamed: 0) 제거하고 데이터 불러오기
homeshop_rating = pd.read_csv('C:/Users/Playdata/2020_Bigcontest_working/dataset/homeshoping_rating.csv').iloc[:, 1:]
homeshop_rating

,date,exp_mins,mom_code,pd_code,pd_name,pd_group,pd_price,sales,weekdays,seasons,rating
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,Tuesday,1,0.0000
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,Tuesday,1,0.0000
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,Tuesday,1,0.0000
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,Tuesday,1,0.0000
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,Tuesday,1,0.0000
...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,Tuesday,1,0.0218
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,Wednesday,1,0.0278
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,Wednesday,1,0.0278
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,Wednesday,1,0.0278


In [3]:
weather = pd.read_csv('C:/Users/Playdata/2020_Bigcontest_working/dataset/weather/weather_final.csv')
weather

,date,temp,rain,humidity,snow,dust,is_rain,is_snow
0,2019-01-01 00:00:00,-5.5,NaN,54.0,NaN,59.0,0,0
1,2019-01-01 01:00:00,-5.9,NaN,56.0,NaN,58.0,0,0
2,2019-01-01 02:00:00,-6.5,NaN,60.0,NaN,44.0,0,0
3,2019-01-01 03:00:00,-6.9,NaN,62.0,NaN,48.0,0,0
4,2019-01-01 04:00:00,-7.2,NaN,57.0,NaN,51.0,0,0
...,...,...,...,...,...,...,...,...
8756,2019-12-31 20:00:00,-7.0,NaN,41.0,NaN,27.0,0,0
8757,2019-12-31 21:00:00,-7.4,NaN,43.0,NaN,27.0,0,0
8758,2019-12-31 22:00:00,-8.2,NaN,42.0,NaN,29.0,0,0
8759,2019-12-31 23:00:00,-7.4,NaN,45.0,NaN,24.0,0,0


- 결합을 해주기 전에 각 데이터 셋의 변수 타입을 확인해봐야 한다.
    - 조인키로 사용할 date 변수의 변수 타입을 동일하게 해주어야 하기 때문!!

In [4]:
homeshop_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37372 entries, 0 to 37371
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      37372 non-null  object 
 1   exp_mins  37372 non-null  float64
 2   mom_code  37372 non-null  int64  
 3   pd_code   37372 non-null  int64  
 4   pd_name   37372 non-null  object 
 5   pd_group  37372 non-null  object 
 6   pd_price  37372 non-null  int64  
 7   sales     37372 non-null  float64
 8   weekdays  37372 non-null  object 
 9   seasons   37372 non-null  int64  
 10  rating    37372 non-null  float64
dtypes: float64(3), int64(4), object(4)
memory usage: 3.1+ MB


In [5]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8761 entries, 0 to 8760
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      8761 non-null   object 
 1   temp      8761 non-null   float64
 2   rain      824 non-null    float64
 3   humidity  8761 non-null   float64
 4   snow      82 non-null     float64
 5   dust      8200 non-null   float64
 6   is_rain   8761 non-null   int64  
 7   is_snow   8761 non-null   int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 547.7+ KB


- 각 데이터 셋에서 date 변수를 활용해서 **"년-월-일-시간" 형태의 새로운 변수(new_date)**를 만들어주겠다.
    - 그 다음, new_date 변수를 조인키로 결합을 시켜줄 것이다.
- 그러기 위해서 먼저, 각 데이터 셋의 date 변수의 변수 타입을 datetime으로 변경해주도록 하겠다.

In [6]:
homeshop_rating['date'] = pd.to_datetime(homeshop_rating['date'], format =  '%Y-%m-%d %H:%M:%S')
weather['date'] = pd.to_datetime(weather['date'], format =  '%Y-%m-%d %H:%M:%S')

In [7]:
homeshop_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37372 entries, 0 to 37371
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      37372 non-null  datetime64[ns]
 1   exp_mins  37372 non-null  float64       
 2   mom_code  37372 non-null  int64         
 3   pd_code   37372 non-null  int64         
 4   pd_name   37372 non-null  object        
 5   pd_group  37372 non-null  object        
 6   pd_price  37372 non-null  int64         
 7   sales     37372 non-null  float64       
 8   weekdays  37372 non-null  object        
 9   seasons   37372 non-null  int64         
 10  rating    37372 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(4), object(3)
memory usage: 3.1+ MB


In [8]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8761 entries, 0 to 8760
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      8761 non-null   datetime64[ns]
 1   temp      8761 non-null   float64       
 2   rain      824 non-null    float64       
 3   humidity  8761 non-null   float64       
 4   snow      82 non-null     float64       
 5   dust      8200 non-null   float64       
 6   is_rain   8761 non-null   int64         
 7   is_snow   8761 non-null   int64         
dtypes: datetime64[ns](1), float64(5), int64(2)
memory usage: 547.7 KB


In [9]:
# 년(year), 월(month), 일(day), 시간(hour) 변수 생성
homeshop_rating['year'] = homeshop_rating['date'].dt.year
homeshop_rating['month'] = homeshop_rating['date'].dt.month
homeshop_rating['day'] = homeshop_rating['date'].dt.day
homeshop_rating['hour'] = homeshop_rating['date'].dt.hour

weather['year'] = weather['date'].dt.year
weather['month'] = weather['date'].dt.month
weather['day'] = weather['date'].dt.day
weather['hour'] = weather['date'].dt.hour

In [10]:
# 아래의 for문을 돌리기 위해 임시적으로 "0" 값으로 채워진 new_date 변수를 미리 만들어 주었음
homeshop_rating['new_date'] = 0

# "년-월-일-시간" 형태의 새로운 변수 new_date 생성
for i in tqdm(range(len(homeshop_rating))):
    homeshop_rating['new_date'][i] = str(homeshop_rating['year'][i]) + '-' + str(homeshop_rating['month'][i]) + '-' + \
                                     str(homeshop_rating['day'][i]) + '-' + str(homeshop_rating['hour'][i])
homeshop_rating

C:\Users\Playdata\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Playdata\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,date,exp_mins,mom_code,pd_code,pd_name,pd_group,pd_price,sales,weekdays,seasons,rating,year,month,day,hour,new_date
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,Tuesday,1,0.0000,2019,1,1,6,2019-1-1-6
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,Tuesday,1,0.0000,2019,1,1,6,2019-1-1-6
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,Tuesday,1,0.0000,2019,1,1,6,2019-1-1-6
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,Tuesday,1,0.0000,2019,1,1,6,2019-1-1-6
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,Tuesday,1,0.0000,2019,1,1,6,2019-1-1-6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,Tuesday,1,0.0218,2019,12,31,23,2019-12-31-23
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,Wednesday,1,0.0278,2020,1,1,0,2020-1-1-0
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,Wednesday,1,0.0278,2020,1,1,0,2020-1-1-0
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,Wednesday,1,0.0278,2020,1,1,0,2020-1-1-0


In [11]:
# 아래의 for문을 돌리기 위해 임시적으로 "0" 값으로 채워진 new_date 변수를 미리 만들어 주었음
weather['new_date'] = 0

# "년-월-일-시간" 형태의 새로운 변수 new_date 생성
for i in tqdm(range(len(weather))):
    weather['new_date'][i] = str(weather['year'][i]) + '-' + str(weather['month'][i]) + '-' + \
                             str(weather['day'][i]) + '-' + str(weather['hour'][i])
weather

C:\Users\Playdata\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\Playdata\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,date,temp,rain,humidity,snow,dust,is_rain,is_snow,year,month,day,hour,new_date
0,2019-01-01 00:00:00,-5.5,NaN,54.0,NaN,59.0,0,0,2019,1,1,0,2019-1-1-0
1,2019-01-01 01:00:00,-5.9,NaN,56.0,NaN,58.0,0,0,2019,1,1,1,2019-1-1-1
2,2019-01-01 02:00:00,-6.5,NaN,60.0,NaN,44.0,0,0,2019,1,1,2,2019-1-1-2
3,2019-01-01 03:00:00,-6.9,NaN,62.0,NaN,48.0,0,0,2019,1,1,3,2019-1-1-3
4,2019-01-01 04:00:00,-7.2,NaN,57.0,NaN,51.0,0,0,2019,1,1,4,2019-1-1-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8756,2019-12-31 20:00:00,-7.0,NaN,41.0,NaN,27.0,0,0,2019,12,31,20,2019-12-31-20
8757,2019-12-31 21:00:00,-7.4,NaN,43.0,NaN,27.0,0,0,2019,12,31,21,2019-12-31-21
8758,2019-12-31 22:00:00,-8.2,NaN,42.0,NaN,29.0,0,0,2019,12,31,22,2019-12-31-22
8759,2019-12-31 23:00:00,-7.4,NaN,45.0,NaN,24.0,0,0,2019,12,31,23,2019-12-31-23


- 다음으로 "년-월-일" 형태의 new_date2 변수를 만들어주도록 하겠다.
    - **밑에서 rain, snow, dust 변수에 대한 결측치들을 처리해 줄 때 필요!!**

In [12]:
# 아래의 for문을 돌리기 위해 임시적으로 "0" 값으로 채워진 new_date2 변수를 미리 만들어 주었음
weather['new_date2'] = 0

# "년-월-일-시간" 형태의 새로운 변수 new_date 생성
for i in tqdm(range(len(weather))):
    weather['new_date2'][i] = str(weather['year'][i]) + '-' + str(weather['month'][i]) + '-' + str(weather['day'][i])
weather

C:\Users\Playdata\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Playdata\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,date,temp,rain,humidity,snow,dust,is_rain,is_snow,year,month,day,hour,new_date,new_date2
0,2019-01-01 00:00:00,-5.5,NaN,54.0,NaN,59.0,0,0,2019,1,1,0,2019-1-1-0,2019-1-1
1,2019-01-01 01:00:00,-5.9,NaN,56.0,NaN,58.0,0,0,2019,1,1,1,2019-1-1-1,2019-1-1
2,2019-01-01 02:00:00,-6.5,NaN,60.0,NaN,44.0,0,0,2019,1,1,2,2019-1-1-2,2019-1-1
3,2019-01-01 03:00:00,-6.9,NaN,62.0,NaN,48.0,0,0,2019,1,1,3,2019-1-1-3,2019-1-1
4,2019-01-01 04:00:00,-7.2,NaN,57.0,NaN,51.0,0,0,2019,1,1,4,2019-1-1-4,2019-1-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8756,2019-12-31 20:00:00,-7.0,NaN,41.0,NaN,27.0,0,0,2019,12,31,20,2019-12-31-20,2019-12-31
8757,2019-12-31 21:00:00,-7.4,NaN,43.0,NaN,27.0,0,0,2019,12,31,21,2019-12-31-21,2019-12-31
8758,2019-12-31 22:00:00,-8.2,NaN,42.0,NaN,29.0,0,0,2019,12,31,22,2019-12-31-22,2019-12-31
8759,2019-12-31 23:00:00,-7.4,NaN,45.0,NaN,24.0,0,0,2019,12,31,23,2019-12-31-23,2019-12-31


- 결합을 위한 조인키(new_date)를 만들어주었으므로, 이제 불필요한 변수인 **"year, month, day, hour" 변수는 제거**하도록 하겠다.
    - 추가적으로 **date라는 변수가 두 개의 데이터 셋에 모두 존재**하므로, **weather 데이터 셋의 date 변수는 제거**해주겠다.

In [13]:
homeshop_rating.drop(['year', 'month', 'day', 'hour'], axis = 1, inplace = True)
weather.drop(['date', 'year', 'month', 'day', 'hour'], axis = 1, inplace = True)

In [14]:
homeshop_rating.head()

,date,exp_mins,mom_code,pd_code,pd_name,pd_group,pd_price,sales,weekdays,seasons,rating,new_date
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,Tuesday,1,0.0,2019-1-1-6
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,Tuesday,1,0.0,2019-1-1-6
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,Tuesday,1,0.0,2019-1-1-6
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,Tuesday,1,0.0,2019-1-1-6
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,Tuesday,1,0.0,2019-1-1-6


In [15]:
weather.head()

,temp,rain,humidity,snow,dust,is_rain,is_snow,new_date,new_date2
0,-5.5,NaN,54.0,NaN,59.0,0,0,2019-1-1-0,2019-1-1
1,-5.9,NaN,56.0,NaN,58.0,0,0,2019-1-1-1,2019-1-1
2,-6.5,NaN,60.0,NaN,44.0,0,0,2019-1-1-2,2019-1-1
3,-6.9,NaN,62.0,NaN,48.0,0,0,2019-1-1-3,2019-1-1
4,-7.2,NaN,57.0,NaN,51.0,0,0,2019-1-1-4,2019-1-1


- (중복된 값을 제외한) 두 데이터 셋의 new_date 개수가 서로 불일치 하기 때문에, **inner join**을 통해 "홈쇼핑+시청률" 데이터 셋에 존재하는 new_date에 대해서만 결합을 수행해주도록 하겠다.
    - 즉, **new_date 변수를 조인키로 inner join** 해주겠다!!

In [16]:
df_shop_rating_weather = pd.merge(homeshop_rating, weather, how = 'inner', on = 'new_date')
df_shop_rating_weather

,date,exp_mins,mom_code,pd_code,pd_name,pd_group,pd_price,sales,weekdays,seasons,rating,new_date,temp,rain,humidity,snow,dust,is_rain,is_snow,new_date2
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,Tuesday,1,0.0000,2019-1-1-6,-7.9,NaN,60.0,NaN,65.0,0,0,2019-1-1
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,Tuesday,1,0.0000,2019-1-1-6,-7.9,NaN,60.0,NaN,65.0,0,0,2019-1-1
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,Tuesday,1,0.0000,2019-1-1-6,-7.9,NaN,60.0,NaN,65.0,0,0,2019-1-1
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,Tuesday,1,0.0000,2019-1-1-6,-7.9,NaN,60.0,NaN,65.0,0,0,2019-1-1
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,Tuesday,1,0.0000,2019-1-1-6,-7.9,NaN,60.0,NaN,65.0,0,0,2019-1-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,Tuesday,1,0.0218,2019-12-31-23,-7.4,NaN,45.0,NaN,24.0,0,0,2019-12-31
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,Wednesday,1,0.0278,2020-1-1-0,-6.5,0.0,38.0,NaN,27.0,0,0,2020-1-1
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,Wednesday,1,0.0278,2020-1-1-0,-6.5,0.0,38.0,NaN,27.0,0,0,2020-1-1
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,Wednesday,1,0.0278,2020-1-1-0,-6.5,0.0,38.0,NaN,27.0,0,0,2020-1-1


# df_shop_rating_weather 데이터 셋에 소비자심지리수 데이터 결합

In [17]:
df_csi = pd.read_csv('C:/Users/Playdata/2020_Bigcontest_working/dataset/CSI/2019_CSI_final.csv')
df_csi

,date,st_living_csi,eco_judgment_csi,ccsi
0,2019-01,90,65,98
1,2019-02,93,70,100
2,2019-03,91,70,100
3,2019-04,93,74,102
4,2019-05,91,69,98
5,2019-06,91,69,98
6,2019-07,91,67,96
7,2019-08,90,63,92
8,2019-09,92,68,97
9,2019-10,92,72,99


- date 변수를 조인키로 사용할 것인데, df_shop_rating_weather 데이터 셋에 이미 date라는 변수가 존재하므로 변수명을 변경해주겠다.
    - new_date3으로 변경!!

In [18]:
df_csi.rename(columns = {'date':'new_date3'}, inplace = True)
df_csi.head()

,new_date3,st_living_csi,eco_judgment_csi,ccsi
0,2019-01,90,65,98
1,2019-02,93,70,100
2,2019-03,91,70,100
3,2019-04,93,74,102
4,2019-05,91,69,98


- 마찬가지로 df_shop_rating_weather 데이터 셋에서도 date 변수를 활용하여 new_date3 변수를 만들어주도록 하겠다.
    - df_csi 데이터 셋과 동일한 format으로 생성!!
- 먼저 datetime 타입으로 변경했던 date 변수를 다시 object 타입으로 변환해주겠다.
    - 그래야 문자열 slicing이 가능하기 때문!!

In [19]:
df_shop_rating_weather['date'] = df_shop_rating_weather['date'].astype(str)
df_shop_rating_weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37372 entries, 0 to 37371
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       37372 non-null  object 
 1   exp_mins   37372 non-null  float64
 2   mom_code   37372 non-null  int64  
 3   pd_code    37372 non-null  int64  
 4   pd_name    37372 non-null  object 
 5   pd_group   37372 non-null  object 
 6   pd_price   37372 non-null  int64  
 7   sales      37372 non-null  float64
 8   weekdays   37372 non-null  object 
 9   seasons    37372 non-null  int64  
 10  rating     37372 non-null  float64
 11  new_date   37372 non-null  object 
 12  temp       37372 non-null  float64
 13  rain       3489 non-null   float64
 14  humidity   37372 non-null  float64
 15  snow       368 non-null    float64
 16  dust       34952 non-null  float64
 17  is_rain    37372 non-null  int64  
 18  is_snow    37372 non-null  int64  
 19  new_date2  37372 non-null  object 
dtypes: flo

In [20]:
# 아래의 for문을 돌리기 위해 임시적으로 "0" 값으로 채워진 new_date3 변수를 미리 만들어 주었음
df_shop_rating_weather['new_date3'] = 0

for i in tqdm(range(len(df_shop_rating_weather))):
    df_shop_rating_weather['new_date3'][i] = df_shop_rating_weather['date'][i][:7]
df_shop_rating_weather['new_date3']

C:\Users\Playdata\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Playdata\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


0        2019-01
1        2019-01
2        2019-01
3        2019-01
4        2019-01
          ...   
37367    2019-12
37368    2020-01
37369    2020-01
37370    2020-01
37371    2020-01
Name: new_date3, Length: 37372, dtype: object

- 이제 두 개의 데이터 셋을 new_date3 변수를 조인키로 사용하여 결합해주도록 하겠다.
    - Inner join

In [21]:
df_final = pd.merge(df_shop_rating_weather, df_csi, how = 'inner', on = 'new_date3')
df_final

,date,exp_mins,mom_code,pd_code,pd_name,pd_group,pd_price,sales,weekdays,seasons,...,humidity,snow,dust,is_rain,is_snow,new_date2,new_date3,st_living_csi,eco_judgment_csi,ccsi
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,Tuesday,1,...,60.0,NaN,65.0,0,0,2019-1-1,2019-01,90,65,98
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,Tuesday,1,...,60.0,NaN,65.0,0,0,2019-1-1,2019-01,90,65,98
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,Tuesday,1,...,60.0,NaN,65.0,0,0,2019-1-1,2019-01,90,65,98
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,Tuesday,1,...,60.0,NaN,65.0,0,0,2019-1-1,2019-01,90,65,98
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,Tuesday,1,...,60.0,NaN,65.0,0,0,2019-1-1,2019-01,90,65,98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,Tuesday,1,...,45.0,NaN,24.0,0,0,2019-12-31,2019-12,92,74,101
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,Wednesday,1,...,38.0,NaN,27.0,0,0,2020-1-1,2020-01,93,78,104
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,Wednesday,1,...,38.0,NaN,27.0,0,0,2020-1-1,2020-01,93,78,104
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,Wednesday,1,...,38.0,NaN,27.0,0,0,2020-1-1,2020-01,93,78,104


- 위 결과를 보면, inner join이 잘 이루어진 것을 알 수 있다.
    - 즉, **df_final 데이터 셋**은 **"매출 실적 + 시청률 + 기상 + 소비자심리지수"** 이렇게 4개의 데이터 셋을 결합해놓은 것!!

# 결측치 처리

## dust 변수 결측치 처리
1. 결측값 전후의 미세먼지 농도 값들을 평균내서 넣어주겠다.
2. 만약 결측값 전후의 미세먼지 농도 값들 중, 결측값이 하나라도 존재하는 경우에는 해당 일자(new_date2)의 평균 미세먼지 농도 값으로 넣어주겠다.
    - ex) "2019년 1월 4일 9시"와 "2019년 1월 4일 11시" 둘 중 하나라도 미세먼지 농도(dust)가 결측치이면, "2019년 1월 4일 10시"의 미세먼지 농도를 "2019년 1월 4일" 미세먼지 농도(dust)의 평균 값으로 대체
3. 만약 해당 일자(new_date2)의 모든 시간대의 미세먼지 농도가 결측이면, 2019년 전체 미세먼지 농도의 평균 값으로 넣어주겠다.
    - ex) 2019년 9월 1일 모든 시간대에 미세먼지 농도(dust)가 결측치이면, 2019년 전체 미세먼지 농도(dust)의 평균 값으로 대체

In [22]:
def fillna_dust(var):
    # 결측값들이 존재하는 index 확인
    null_index = df_final[df_final[var].isnull()].index

    for idx in null_index:
        # 1. 결측값 전후의 미세먼지 농도 값들의 평균 값으로 대체
        mean_dust = round((df_final.loc[idx - 1, var] + df_final.loc[idx + 1, var]) / 2) # 정수로 반올림
        df_final.loc[df_final.index == idx, var] = mean_dust
    # 결측치들이 잘 채워졌는지 확인
    print('존재하는 결측값 개수: ', df_final[var].isnull().sum())

    g = df_final[var].groupby(df_final['new_date2'])
    df_mean_var = pd.DataFrame(g.mean()).reset_index()
    df_mean_var.rename(columns = {var:'mean_{}'.format(var)}, inplace = True)

    df_mean_var['mean_{}'.format(var)] = round(df_mean_var['mean_{}'.format(var)]) # 정수로 반올림

    df_final2 = pd.merge(df_final, df_mean_var, how = 'inner', on = 'new_date2')

    # 2. 결측값 전후의 미세먼지 농도 값들 중, 
    # 하나라도 결측이 존재하는 경우에는 해당 일자(new_date2)의 평균 미세먼지 농도 값(mean_dust)으로 대체
    df_final2[var] = df_final2[var].fillna(df_final2['mean_{}'.format(var)])
    # 결측치들이 잘 채워졌는지 확인
    print('존재하는 결측값 개수: ', df_final2[var].isnull().sum())

    # 3. 해당 일자(new_date2)의 모든 시간대의 미세먼지 농도가 결측인 경우, 
    # 2019년 전체 미세먼지 농도의 평균 값으로 대체
    df_final2[var] = df_final2[var].fillna(round(np.mean(df_final2[var]))) # 정수로 반올림
    # 결측치들이 잘 채워졌는지 확인
    print('존재하는 결측값 개수: ', df_final2[var].isnull().sum())
    
    # 모든 작업이 끝난 데이터 셋을 csv 파일로 내보내기
    df_final2.to_csv('C:/Users/Playdata/2020_Bigcontest_working/dataset/df_fillna_dust.csv', index = False)

In [23]:
fillna_dust('dust')

존재하는 결측값 개수:  2420
존재하는 결측값 개수:  641
존재하는 결측값 개수:  0


- 미세먼지 농도(dust) 변수에 대한 결측치 처리는 잘 수행되었다.

## rain 변수 결측치 처리
- dust 변수와 다르게, rain 변수는 0번째 index에 결측값이 존재한다.
- 따라서 먼저 0번째 index에 대해서 결측치를 처리해 준 뒤에 dust 변수와 동일한 방법으로 진행해주겠다.
    - rain 변수의 0번째 index의 결측값에 대해서는 각각 2019년 rain 변수의 전체 평균 값으로 대체해주겠다.

- 먼저 위에서 dust 변수에 대한 결측치를 처리해 준 데이터 셋을 불러오겠다.
    - 단, 편리성을 위해 불러온 파일명을 df_final로 해주겠다.

In [24]:
df_final = pd.read_csv('C:/Users/Playdata/2020_Bigcontest_working/dataset/df_fillna_dust.csv')
df_final.isnull().sum() # 불러온 데이터 셋의 dust 변수 결측치 처리가 잘 이루어졌는지 확인

date                    0
exp_mins                0
mom_code                0
pd_code                 0
pd_name                 0
pd_group                0
pd_price                0
sales                   0
weekdays                0
seasons                 0
rating                  0
new_date                0
temp                    0
rain                33883
humidity                0
snow                37004
dust                    0
is_rain                 0
is_snow                 0
new_date2               0
new_date3               0
st_living_csi           0
eco_judgment_csi        0
ccsi                    0
mean_dust             641
dtype: int64

In [25]:
null_rain_index = df_final[df_final['rain'].isnull()].index
null_rain_index

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            37358, 37359, 37360, 37361, 37362, 37363, 37364, 37365, 37366,
            37367],
           dtype='int64', length=33883)

In [26]:
def fillna_rain(var):
    # 0번째 index에 존재하는 결측값은 2019년 rain 변수의 전체 평균 값으로 대체
    df_final.loc[df_final.index == 0, var] = round(np.mean(df_final[var])) # 정수로 반올림
    # 결측값들이 존재하는 index 확인
    null_index = df_final[df_final[var].isnull()].index

    for idx in null_index:
        # 1. 결측값 전후의 강수량 값들의 평균 값으로 대체
        mean_rain = round((df_final.loc[idx - 1, var] + df_final.loc[idx + 1, var]) / 2) # 정수로 반올림
        df_final.loc[df_final.index == idx, var] = mean_rain
    # 결측치들이 잘 채워졌는지 확인
    print('존재하는 결측값 개수: ', df_final[var].isnull().sum())

    g = df_final[var].groupby(df_final['new_date2'])
    df_mean_var = pd.DataFrame(g.mean()).reset_index()
    df_mean_var.rename(columns = {var:'mean_{}'.format(var)}, inplace = True)

    df_mean_var['mean_{}'.format(var)] = round(df_mean_var['mean_{}'.format(var)]) # 정수로 반올림

    df_final2 = pd.merge(df_final, df_mean_var, how = 'inner', on = 'new_date2')

    # 2. 결측값 전후의 강수량 값들 중, 
    # 하나라도 결측이 존재하는 경우에는 해당 일자(new_date2)의 평균 강수량 값(mean_rain)으로 대체
    df_final2[var] = df_final2[var].fillna(df_final2['mean_{}'.format(var)])
    # 결측치들이 잘 채워졌는지 확인
    print('존재하는 결측값 개수: ', df_final2[var].isnull().sum())

    # 3. 해당 일자(new_date2)의 모든 시간대의 강수량이 결측인 경우, 
    # 2019년 전체 강수량의 평균 값으로 대체
    df_final2[var] = df_final2[var].fillna(round(np.mean(df_final2[var]))) # 정수로 반올림
    # 결측치들이 잘 채워졌는지 확인
    print('존재하는 결측값 개수: ', df_final2[var].isnull().sum())
    
    # 모든 작업이 끝난 데이터 셋을 csv 파일로 내보내기
    df_final2.to_csv('C:/Users/Playdata/2020_Bigcontest_working/dataset/df_fillna_dust_rain.csv', index = False)

In [27]:
fillna_rain('rain')

존재하는 결측값 개수:  33882
존재하는 결측값 개수:  23567
존재하는 결측값 개수:  0


## snow 변수 결측치 처리

- 먼저 위에서 rain 변수에 대한 결측치를 처리해 준 데이터 셋을 불러오겠다.
    - 마찬가지로 편리성을 위해 불러온 파일명을 df_final로 해주겠다.

In [28]:
df_final = pd.read_csv('C:/Users/Playdata/2020_Bigcontest_working/dataset/df_fillna_dust_rain.csv')
df_final.isnull().sum() # 불러온 데이터 셋의 rain 변수 결측치 처리가 잘 이루어졌는지 확인

date                    0
exp_mins                0
mom_code                0
pd_code                 0
pd_name                 0
pd_group                0
pd_price                0
sales                   0
weekdays                0
seasons                 0
rating                  0
new_date                0
temp                    0
rain                    0
humidity                0
snow                37004
dust                    0
is_rain                 0
is_snow                 0
new_date2               0
new_date3               0
st_living_csi           0
eco_judgment_csi        0
ccsi                    0
mean_dust             641
mean_rain           23567
dtype: int64

In [29]:
len(df_final)

37372

In [30]:
df_final['snow'].isnull().sum()

37004

- 위 결과를 보면 알 수 있듯이, **총 37372개의 자료 중, 37003개의 결측값들이 존재**한다.
    - 때문에 snow 변수의 결측값들을 rain 변수와 동일한 방법으로 처리해주는 것은 무의미하다고 생각한다.
- 따라서 **snow 변수에 대한 결측치 처리는 따로 해주지 않을 것**이며, 추후 예측 모델 생성 시에는 앞서 만들어놓은 is_snow 변수를 사용하도록 하겠다.

# 불필요한 변수 제거
- 이제 최종 데이터 셋에서 불필요한 변수들은 제거해주도록 하겠다.
    - new_date 변수 
    - new_date2 변수
    - new_date3 변수
    - mean_dust 변수
    - mean_rain 변수

In [31]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37372 entries, 0 to 37371
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              37372 non-null  object 
 1   exp_mins          37372 non-null  float64
 2   mom_code          37372 non-null  int64  
 3   pd_code           37372 non-null  int64  
 4   pd_name           37372 non-null  object 
 5   pd_group          37372 non-null  object 
 6   pd_price          37372 non-null  int64  
 7   sales             37372 non-null  float64
 8   weekdays          37372 non-null  object 
 9   seasons           37372 non-null  int64  
 10  rating            37372 non-null  float64
 11  new_date          37372 non-null  object 
 12  temp              37372 non-null  float64
 13  rain              37372 non-null  float64
 14  humidity          37372 non-null  float64
 15  snow              368 non-null    float64
 16  dust              37372 non-null  float6

In [32]:
df_final.drop(['new_date', 'new_date2', 'new_date3', 'mean_dust', 'mean_rain'], axis = 1, inplace = True)
df_final

,date,exp_mins,mom_code,pd_code,pd_name,pd_group,pd_price,sales,weekdays,seasons,...,temp,rain,humidity,snow,dust,is_rain,is_snow,st_living_csi,eco_judgment_csi,ccsi
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,Tuesday,1,...,-7.9,1.0,60.0,NaN,65.0,0,0,90,65,98
1,2019-01-01 06:00:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,Tuesday,1,...,-7.9,1.0,60.0,NaN,65.0,0,0,90,65,98
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,Tuesday,1,...,-7.9,1.0,60.0,NaN,65.0,0,0,90,65,98
3,2019-01-01 06:20:00,20.0,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,Tuesday,1,...,-7.9,1.0,60.0,NaN,65.0,0,0,90,65,98
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,Tuesday,1,...,-7.9,1.0,60.0,NaN,65.0,0,0,90,65,98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,20.0,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,Tuesday,1,...,-7.4,0.0,45.0,NaN,24.0,0,0,92,74,101
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,Wednesday,1,...,-6.5,0.0,38.0,NaN,27.0,0,0,93,78,104
37369,2020-01-01 00:00:00,20.0,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,Wednesday,1,...,-6.5,0.0,38.0,NaN,27.0,0,0,93,78,104
37370,2020-01-01 00:00:00,20.0,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,Wednesday,1,...,-6.5,0.0,38.0,NaN,27.0,0,0,93,78,104


# 최종 결합된 데이터 셋 csv 파일로 내보내기

In [33]:
df_final.to_csv('C:/Users/Playdata/2020_Bigcontest_working/dataset/df_train_final.csv', index = False)